In [1]:
# Import dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from citipy import citipy
import requests
import time
from datetime import datetime

In [2]:
# Creat 1500 random latitude and longitude combinations
lats = np.random.uniform(-90, 90, size=150)
lngs = np.random.uniform(-180, 180, size=150)
lat_lngs = zip(lats, lngs)
coordinates = list(lat_lngs)

In [3]:
# Create a list of cities' name
cities = []

# Indentify the nearest cities for the coordinates
for i in coordinates:
    city = citipy.nearest_city(i[0], i[1]).city_name
    
    # Add city to the list
    if city not in cities:
        cities.append(city)
        
# Show how many cities we detect
len(cities)

114

In [4]:
# Import the API key
from config import weather_api_key

In [5]:
url = f'http://api.openweathermap.org/data/2.5/weather?appid={weather_api_key}&q='

In [21]:
# Create an empty list to hold the weather data
city_data = []

# Illustrate logging beginning
print('Beginning of Data Retrieval\n***************************')

# Create Counters
record_count = 1
set_count = 1

# Loop through all the cities in our list
for i, city in enumerate(cities):
    
    # Group cities in sets of 50 for logging
    if (i%50 == 0 and i >=50):
        set_count += 1
        record_count = 1
        time.sleep(60)
        
    # Create URL for each city and replace blanks with + for city names with white spaces
    city_url = url + city.replace(' ','+')

    # Log the URL, and other information
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    
    # Count the records
    record_count += 1
    
    # API request for each city
    try:
        # Retrieve data from JSON
        city_weather = requests.get(city_url).json()
        
        # Extract required field of data
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        
        # Convert time from UTC to ISO
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        
        # Append extracted data to city_data
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})
        
    # Code the except block
    except:
        print('City not found')
        pass
    
# Show data extraction is completed
print('End of Data Retrieval\n***********************')

Beginning of Data Retrieval
***************************
Processing Record 1 of Set 1 | hobart
Processing Record 2 of Set 1 | hermanus
Processing Record 3 of Set 1 | albany
Processing Record 4 of Set 1 | macusani
Processing Record 5 of Set 1 | iqaluit
Processing Record 6 of Set 1 | rikitea
Processing Record 7 of Set 1 | tuban
Processing Record 8 of Set 1 | khilok
Processing Record 9 of Set 1 | los llanos de aridane
Processing Record 10 of Set 1 | guhagar
Processing Record 11 of Set 1 | klaksvik
Processing Record 12 of Set 1 | nikolskoye
Processing Record 13 of Set 1 | ratangarh
Processing Record 14 of Set 1 | jamestown
Processing Record 15 of Set 1 | butaritari
Processing Record 16 of Set 1 | norman wells
Processing Record 17 of Set 1 | vila franca do campo
Processing Record 18 of Set 1 | satitoa
City not found
Processing Record 19 of Set 1 | tasiilaq
Processing Record 20 of Set 1 | arraial do cabo
Processing Record 21 of Set 1 | airai
Processing Record 22 of Set 1 | kruisfontein
Proces

In [23]:
# Create a DataFrame from the extracted data
city_data_df = pd.DataFrame(city_data)
city_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Hobart,-42.8794,147.3294,282.18,93,100,2.57,AU,2022-09-28 17:20:13
1,Hermanus,-34.4187,19.2345,287.48,77,63,1.95,ZA,2022-09-28 17:20:13
2,Albany,42.6001,-73.9662,290.88,70,82,1.34,US,2022-09-28 17:15:41
3,Macusani,-14.0833,-70.4333,284.49,35,15,3.49,PE,2022-09-28 17:20:14
4,Iqaluit,63.7506,-68.5145,276.00,60,75,6.17,CA,2022-09-28 17:20:14
...,...,...,...,...,...,...,...,...,...
100,Husavik,66.0449,-17.3389,280.52,74,100,1.07,IS,2022-09-28 17:23:06
101,Montes Claros,-16.7350,-43.8617,307.03,22,40,4.63,BR,2022-09-28 17:21:12
102,Kavieng,-2.5744,150.7967,301.03,70,99,6.66,PG,2022-09-28 17:23:07
103,Bela,25.9333,81.9833,300.18,68,0,0.72,IN,2022-09-28 17:23:08


In [25]:
# Reorder the columns
city_data_df = city_data_df[['City','Country','Date','Lat','Lng','Max Temp','Humidity','Cloudiness','Wind Speed']]
city_data_df

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Hobart,AU,2022-09-28 17:20:13,-42.8794,147.3294,282.18,93,100,2.57
1,Hermanus,ZA,2022-09-28 17:20:13,-34.4187,19.2345,287.48,77,63,1.95
2,Albany,US,2022-09-28 17:15:41,42.6001,-73.9662,290.88,70,82,1.34
3,Macusani,PE,2022-09-28 17:20:14,-14.0833,-70.4333,284.49,35,15,3.49
4,Iqaluit,CA,2022-09-28 17:20:14,63.7506,-68.5145,276.00,60,75,6.17
...,...,...,...,...,...,...,...,...,...
100,Husavik,IS,2022-09-28 17:23:06,66.0449,-17.3389,280.52,74,100,1.07
101,Montes Claros,BR,2022-09-28 17:21:12,-16.7350,-43.8617,307.03,22,40,4.63
102,Kavieng,PG,2022-09-28 17:23:07,-2.5744,150.7967,301.03,70,99,6.66
103,Bela,IN,2022-09-28 17:23:08,25.9333,81.9833,300.18,68,0,0.72


In [27]:
# Create an output CSV file and export the data
output_file_path = 'Weather_data/cities.csv'
city_data_df.to_csv(output_file_path, index_label = "City ID")